In [28]:
%matplotlib inline

cf. David Cox, John Little, Donal O’Shea. **Ideals, Varieties, and Algorithms: An Introduction to Computational Algebraic Geometry and Commutative Algebra**, Fourth Edition, Springer, 2015

Same ways to construct Polynomial Rings

In [245]:
PolyQQ_3.<x,y,z>=PolynomialRing(QQ,3)

In [242]:
PolyQQ_3_b = PolynomialRing(QQ,'x,y,z')

In [8]:
PolyQQ_3==PolyQQ_3_b

True

Implementing 

$$
f=2x^3y^2z + \frac{3}{2}y^3z^3 - 3xyz + y^2
$$  

In [246]:
f = 2*x**3*y**2*z+3/2*y**3*z**3-3*x*y*z+y**2

In [247]:
# many (rich amount!) Python methods to try out
print( f.args())
print( f.degree());print(f.degrees());print(f.dict());print(f.exponents());print(f.monomials());print(f.variables())

(x, y, z)
6
(3, 3, 3)
{(3, 2, 1): 2, (1, 1, 1): -3, (0, 2, 0): 1, (0, 3, 3): 3/2}
[(3, 2, 1), (0, 3, 3), (1, 1, 1), (0, 2, 0)]
[x^3*y^2*z, y^3*z^3, x*y*z, y^2]
(x, y, z)


In [249]:
PolyQQ_3.order()

NotImplementedError: 

## Properties of Grobner Bases

In [324]:
Poly_RR__2.<x,y>=PolynomialRing(RR,2,order="lex")

In [327]:
(x**5*y).quo_rem( x**2*y-y**2 )

(x^3 + x*y, x*y^3)

# Buchberger's Algorithm
cf. Sec. 7 of Cox, Little, O'Shea (2015), pp. 90

### Example 1

In [311]:
Poly_QQ__2.<x,y> = PolynomialRing(QQ,2,order="deglex")
f1=x**3-2*x*y
f2=x**2*y-2*y**2+x

In [312]:
make_Spoly(f1,f2).lt()

-x^2

In [314]:
make_Spoly(f1,f2).lt() in Ideal(f1.lt(),f2.lt())

False

In [321]:
print( make_Spoly(f1,f2).lt().quo_rem( f1.lt() ) )
print( make_Spoly(f1,f2).lt().quo_rem( f2.lt() ) )

(0, -x^2)
(0, -x^2)


In [332]:
F=[f1,f2]

In [333]:
F.append( make_Spoly(f1,f2).lt().quo_rem(f1.lt())[1])
print(F)

[x^3 - 2*x*y, x^2*y - 2*y^2 + x, -x^2]


In [334]:
from itertools import combinations
M = [_ for _ in combinations(F,2)]

In [336]:
make_Spoly(M[0][0],M[0][1])

-x^2

In [337]:
make_Spoly(M[1][0],M[1][1])

-2*x*y

In [ ]:
# This is the original implementation from James Carlson, slightly modified
# cf. https://www.math.utah.edu/~carlson/cimat/lecture1.pdf
def div(f,g, polyring):
    zeroelement = polyring(0)
    n = len(g)
    p, r, a = f,0, [0 for x in range(0,n)]
    while p != zeroelement:
        i, divisionoccured = 0, False
        while i < n and divisionoccured == False:
            fp_remainder = (p.lt()).quo_rem(g[i].lt())[0]
            if fp_remainder == zeroelement:
                a[i] += p.lt()//g[i].lt()
                p -= p.lt()//g[i].lt()*g[i]
                divisionoccured == True
            else:
                i += 1
        if divisionoccured == False:
            r += p.lt()
            p -= p.lt()
    return a, r

In [377]:
# I understand my version here slightly better because I stepped through each step with an explicit example below; 
# I'll copy this algorithm here:
def DivisionAlgo( p, fs, ring):
    """ DivisionAlgo - Division algorithm
    """
    s=len(fs)
    zeroelement = ring(0)    
    a=[zeroelement for i in range(s)]
    r=zeroelement
    while p != zeroelement:
        division_FLAG=False
        for i in range(s):
            p_dividedby_fs = p.lm().quo_rem(fs[i].lm())
            if p_dividedby_fs[1] == zeroelement:
                a[i] += p.lt().quo_rem(fs[i].lt())[0]  
#                p -= p.lt().quo_rem(fs[i].lt())[0]*fs[i]
                p -= p.lt()//fs[i].lt()*fs[i]
                division_FLAG=True
                break
        if division_FLAG is False:
            r += p.lt()
            p -= p.lt()
    return r,a

This is *exactly*  
$$  
\overline{S(f_1,f_3)^F}  =-2xy \neq 0
$$  

In [341]:
DivisionAlgo( make_Spoly(M[1][0],M[1][1]), F,Poly_QQ__2)

(-2*x*y, [0, 0, 0])

In [342]:
r,a = DivisionAlgo( make_Spoly(M[1][0],M[1][1]), F,Poly_QQ__2)
F.append(r)

In [350]:
print(F)
M = [_ for _ in combinations(F,2)]
print(DivisionAlgo(make_Spoly(M[0][0],M[0][1]), F, Poly_QQ__2) )
print(DivisionAlgo(make_Spoly(M[1][0],M[1][1]), F, Poly_QQ__2) )
print(DivisionAlgo(make_Spoly(M[2][0],M[2][1]), F, Poly_QQ__2) )
print(DivisionAlgo(make_Spoly(M[3][0],M[3][1]), F, Poly_QQ__2) )


[x^3 - 2*x*y, x^2*y - 2*y^2 + x, -x^2, -2*x*y]
(0, [0, 0, 1, 0])
(0, [0, 0, 0, 1])
(0, [0, 0, 0, y])
(-2*y^2 + x, [0, 0, 0, 0])


"Hence, we must also add $f_5 = -2y^2+x$ to our generating set."  pp. 91, Sec.7 Buchberger's Algorithm

In [351]:
print( make_Spoly(M[3][0],M[3][1]) )
F.append( make_Spoly(M[3][0],M[3][1]) )

-2*y^2 + x


In [378]:
print(F)
M = [_ for _ in combinations(F,2)]
print(DivisionAlgo(make_Spoly(M[0][0],M[0][1]), F, Poly_QQ__2) )
print(DivisionAlgo(make_Spoly(M[1][0],M[1][1]), F, Poly_QQ__2) )
print(DivisionAlgo(make_Spoly(M[2][0],M[2][1]), F, Poly_QQ__2) )
print(DivisionAlgo(make_Spoly(M[3][0],M[3][1]), F, Poly_QQ__2) )
print(DivisionAlgo(make_Spoly(M[4][0],M[4][1]), F, Poly_QQ__2) )
print(DivisionAlgo(make_Spoly(M[5][0],M[5][1]), F, Poly_QQ__2) )
print(DivisionAlgo(make_Spoly(M[6][0],M[6][1]), F, Poly_QQ__2) )
print(DivisionAlgo(make_Spoly(M[7][0],M[7][1]), F, Poly_QQ__2) )
print(DivisionAlgo(make_Spoly(M[8][0],M[8][1]), F, Poly_QQ__2) )
print(DivisionAlgo(make_Spoly(M[9][0],M[9][1]), F, Poly_QQ__2) )


[x^3 - 2*x*y, x^2*y - 2*y^2 + x, -x^2, -2*x*y, -2*y^2 + x]
(0, [0, 0, 1, 0, 0])
(0, [0, 0, 0, 1, 0])
(0, [0, 0, 0, y, 0])
(0, [1/2*x, 1, 0, y^2, -1])
(0, [0, 0, 0, 0, 1])
(0, [0, 0, 0, 0, 1])
(0, [1/2, 0, 0, -1/2, y])
(0, [0, 0, 0, 0, 0])
(0, [1/2, 0, 0, -1/2, 0])
(0, [0, 0, -1/2, 0, 0])


EY:20170311 In the next 2 lines, I had to debug my DivisionAlgo, Division Algorithm implementation.  I wanted to keep this to show that you can use the very useful Python interactive interpreter in jupyter to effectively debug your program/function.

In [366]:
print(DivisionAlgo( make_Spoly( M[4][0],M[4][1] ), [_ for _ in reversed(F)],Poly_QQ__2) )
print(DivisionAlgo( make_Spoly( M[5][0],M[5][1] ), [_ for _ in reversed(F)],Poly_QQ__2) )
print(make_Spoly( M[4][0],M[4][1]))
print(make_Spoly( M[5][0],M[5][1]))


(x, [1, 0, 0, 0, 0])
(x, [1, 0, 0, 0, 0])
-2*y^2 + x
-2*y^2 + x


In [376]:
p=make_Spoly(M[4][0],M[4][1])
r=Poly_QQ__2(0)
print(p)
print( p.lm().quo_rem(F[0].lm()) ); print( p.lm().quo_rem(F[0].lm())[1] == Poly_QQ__2(0) );r+=p.lt(); p -= p.lt();
print(p);print( p.lm().quo_rem(F[1].lm()) ); print( p.lm().quo_rem(F[1].lm())[1] == Poly_QQ__2(0) );r+=p.lt(); p-=p.lt()
print(p);print( p.lm().quo_rem(F[2].lm()) ); print( p.lm().quo_rem(F[2].lm())[1] == Poly_QQ__2(0) );
print(p.lt()//F[2].lt()*F[2])
p-=p.lt()//F[2].lt()*F[2]
# r+=p.lt(); p-=p.lt()
print(p);print( p.lm().quo_rem(F[3].lm()) ); print( p.lm().quo_rem(F[3].lm())[1] == Poly_QQ__2(0) );r+=p.lt(); p-=p.lt()


-2*y^2 + x
(0, y^2)
False
x
(0, x)
False
0
(0, 0)
True
0
0
(0, 0)
True


Theorem 2 (Buchberger's Algorithm) (cf. pp. 91 of Cox, Little, O'Shea (2015))

In [379]:
def BuchbergerAlgo(F,polyring):
    """ BuchbergerAlgo : Buchberger's Algorithm
    INPUT
    =====
    @type F : Python list of Sage Math polynomials
    @param F : F = (f_1, \dots, f_s)
    
    @type polyring : Sage Math PolynomialRing
    """
    zeroelement = polyring(0)
    pairs=[_ for _ in combinations(F,2)]
#    NO_PAIRS = len(pairs)
    G=F
    G_2 = [] # denoted G' or Gprime in Cox, Little, O'Shea (2015)

    # first case guaranteed to run 
    while G_2 != G:
        G_2 = G
        for pair in pairs:
            p=pair[0]
            q=pair[1]
            r,a_list = DivisionAlgo(make_Spoly(p,q), G, polyring)
            if r != zeroelement:
                G.append(r)
        # END of while, break condition being when we don't add,append, anymore to G
    return G

In [381]:
BuchbergerAlgo(F,Poly_QQ__2)




[x^3 - 2*x*y, x^2*y - 2*y^2 + x, -x^2, -2*x*y, -2*y^2 + x]

# From Martin R. Albrecht *DTU Crypto Group*, **Gröbner Bases**
cf. https://martinralbrecht.files.wordpress.com/2010/07/20131022_buchberger_dtu.pdf [Grobner Bases, Albrecht](https://martinralbrecht.files.wordpress.com/2010/07/20131022_buchberger_dtu.pdf)

In [2]:
P.<x,y,z>=PolynomialRing(FiniteField(7))

In [3]:
f=x*y+3*y*z-3

In [4]:
g=x^2-2*y^2

In [5]:
f*g

x^3*y - 2*x*y^3 + 3*x^2*y*z + y^3*z - 3*x^2 - y^2

In [6]:
f+g

x^2 + x*y - 2*y^2 + 3*y*z - 3

In [7]:
P.<x,y>=PolynomialRing(FiniteField(7))

In [8]:
f=2*x*y+3

In [9]:
f.monomials()

[x*y, 1]

In [10]:
f.coefficients()

[2, 3]

## Monomial Orderings

In [11]:
P.<x,y,z>=PolynomialRing(FiniteField(7),order="lex")

In [12]:
x^2*y*z^2 > x*y^3*z

True

In [13]:
P.<x,y,z>=PolynomialRing(FiniteField(7), order="deglex")

In [14]:
x^2*y*z^2 > x*y^3*z

True

In [15]:
P.<x,y,z>=PolynomialRing(FiniteField(7), order="degrevlex")

In [16]:
x^2*y*z^2 > x*y^3*z

False

### Exercises

In [18]:
Poly_F7 = PolynomialRing(FiniteField(7),'x,y,z',order='deglex')

In [41]:
f=4*x**5*y*z**4 + 2*x**2*y**8 - x*y**4 + x*y*z**3+4*y*z+3*x
LatexExpr(f)

2*x^2*y^8 - 3*x^5*y*z^4 - x*y^4 + x*y*z^3 - 3*y*z + 3*x

In [42]:
Poly_F7 = PolynomialRing(FiniteField(7),'x,y,z',order='lex')
f=4*x**5*y*z**4 + 2*x**2*y**8 - x*y**4 + x*y*z**3+4*y*z+3*x
LatexExpr(f)

2*x^2*y^8 - 3*x^5*y*z^4 - x*y^4 + x*y*z^3 - 3*y*z + 3*x

In [43]:
Poly_F7 = PolynomialRing(FiniteField(7),'x,y,z',order='degrevlex')
f=4*x**5*y*z**4 + 2*x**2*y**8 - x*y**4 + x*y*z**3+4*y*z+3*x
LatexExpr(f)

2*x^2*y^8 - 3*x^5*y*z^4 - x*y^4 + x*y*z^3 - 3*y*z + 3*x

## Polynomial Division
### Polynomial Division II 
#### Algorithm 1: Polynomial Division

In [69]:
Poly_F7__2 = PolynomialRing(FiniteField(7),'x,y',order='deglex')
f = 3*x**2*y + 2*y**2+x+1
f1 = 2*x**2+1
f2=y+1

In [70]:
print(p.lm());print(f_1.lm());print(f_2.lm())

x^2*y
x^2
y


In [71]:
p.lm()/f1.lm()

y

In [72]:
print(p-p.lm()/f1.lm()*f1)
#(p-p.lm()/f_1.lm()*f_1)

x^2*y + 2*y^2 + x - y + 1


In [73]:
f==(-2*y)*f1 + (2*y)*f2 + (x+1)

True

I will try to step through the division algorithm

In [78]:
p=f
fs = [f1,f2]
s = len(fs)
a = [0 for i in range(s)]
r=0
division_FLAG = False
p.lm()/fs[0].lm()
print(p.lm()/fs[0].lm())
a[0] = a[0] + p.lt()/fs[0].lt()
print(a[0])
p = p - p.lt()/fs[0].lt()*fs[0]
print(p)
division_FLAG=True

y
-2*y
2*y^2 + x + 2*y + 1


In [80]:
p is not 0
print(p is not 0)
print( p.lm()/fs[0].lm() )

True


AttributeError: 'sage.rings.fraction_field_element.FractionFieldElement' object has no attribute 'lm'

I'll try `quo_rem`

In [82]:
p=f
p.lt().quo_rem( fs[0].lt() )

(-2*y, 0)

In [136]:
p=f
fs = [f1,f2]
s=len(fs)
a=[0 for i in range(s)]
r=0
division_FLAG=False
p_dividedby_fs = p.lm().quo_rem(fs[0].lm())
print( p_dividedby_fs[0])
if p_dividedby_fs[0] is not Poly_F7__2(0):
    a[0] += p.lt().quo_rem(fs[0].lt())[0]
    print(a[0])
    p -= p.lt().quo_rem(fs[0].lt())[0]*fs[0].lt()
    print(p)
    division_FLAG=True
p_dividedby_fs = p.lm().quo_rem(fs[0].lm())
print( p_dividedby_fs[0]  )
print( p_dividedby_fs[0] is not Poly_F7__2(0)) # notice how we need to "FORCE" 0 to belong to \mathbb{F}_7[x,y]   
if p_dividedby_fs[1] is not Poly_F7__2(0):
    a[1] += p.lt().quo_rem(fs[1].lt())[0]
    print(a[1])
    p -= p.lt().quo_rem(fs[1].lt())[0]*fs[1].lt()
    print(p)
    division_FLAG=True
division_FLAG=False
p_dividedby_fs = p.lm().quo_rem(fs[0].lm())
print( p_dividedby_fs)
p_dividedby_fs = p.lm().quo_rem(fs[1].lm())
print( p_dividedby_fs)
if division_FLAG is False:
    r += p.lt()
    print(r)
    p -= p.lt()
    print(p)
print(p is Poly_F7__2(0)) 
print(p == Poly_F7__2(0))
division_FLAG=False
p_dividedby_fs = p.lm().quo_rem(fs[0].lm())
print( p_dividedby_fs)
p_dividedby_fs = p.lm().quo_rem(fs[1].lm())
print( p_dividedby_fs)
if division_FLAG is False:
    r += p.lt()
    print(r)
    p -= p.lt()
    print(p)
print(p is Poly_F7__2(0)) # vs.
print(p == Poly_F7__2(0))

y
-2*y
2*y^2 + x + 1
0
True
2*y
x + 1
(0, x)
(0, x)
x
1
False
False
(0, 1)
(0, 1)
x + 1
0
False
True


In [138]:
def DivisionAlgo( p, fs, ring):
    """ DivisionAlgo - Division algorithm
    """
    s=len(fs)
    zeroelement = ring(0)    
    a=[zeroelement for i in range(s)]
    r=zeroelement
    while p != zeroelement:
        division_FLAG=False
        for i in range(s):
            p_dividedby_fs = p.lm().quo_rem(fs[i].lm())
            if p_dividedby_fs[1] == zeroelement:
                a[i] += p.lt().quo_rem(fs[i].lt())[0]  
                p -= p.lt().quo_rem(fs[i].lt())[0]*fs[i].lt()
                division_FLAG=True
                break
        if division_FLAG is False:
            r += p.lt()
            p -= p.lt()
    return r,a

In [141]:
r_eg, a_eg = DivisionAlgo( f,[f1,f2],Poly_F7__2)
print( r_eg,a_eg)

(x + 1, [-2*y, 2*y])


### Exercises

In [152]:
Poly_F7__2 = PolynomialRing(FiniteField(7),'x,y',order='deglex') # \mathbb{F}_7[x,y] so it has 2 variables
f=x**7*y**2+x**3*y**2-y+1
f1=x*y**2-x
f2=x-y**3

In [153]:
DivisionAlgo(f,[f1,f2],Poly_F7__2)

(-y + 1, [x^6 + x^2, 0])

In [154]:
Poly_F7__2 = PolynomialRing(FiniteField(7),'x,y',order='lex') # \mathbb{F}_7[x,y] so it has 2 variables
f=x**7*y**2+x**3*y**2-y+1
f1=x*y**2-x
f2=x-y**3

In [155]:
DivisionAlgo(f,[f1,f2],Poly_F7__2)

(-y + 1, [x^6 + x^2, 0])

In [156]:
Poly_F7__2 = PolynomialRing(FiniteField(7),'x,y',order='lex') # \mathbb{F}_7[x,y] so it has 2 variables
f=x*y**2-x
f1=x*y+1
f2=y**2-1

In [157]:
DivisionAlgo(f,[f1,f2],Poly_F7__2)

(-x, [y, 0])

In [158]:
DivisionAlgo(f,[f2,f1],Poly_F7__2)

(-x, [x, 0])

In [159]:
Poly_F7__3 = PolynomialRing(FiniteField(7),'x,y,z',order='deglex') # \mathbb{F}_7[x,y,z] so it has 3 variables
f=x*y**2*z**2+x*y-y*z
f1=x-y**2
f2=y-z**3
f3=z**2-1

In [160]:
DivisionAlgo(f,[f1,f2,f3],Poly_F7__3)

(x*y - y*z, [-x*z^2, 0, 0])

In [161]:
DivisionAlgo(f,[f2,f3,f1],Poly_F7__3)

(x*y - y*z, [0, x*y^2, 0])

In [162]:
DivisionAlgo(f,[f3,f1,f2],Poly_F7__3)

(x*y - y*z, [x*y^2, 0, 0])

## Ideals
### Definition

In [163]:
P.<x,y,z> = PolynomialRing(FiniteField(127),order='deglex')

In [164]:
I=ideal(x*y+z,y^3+1,z^2-x*5-1)

In [165]:
(x*y+z)+(y^3+1) in I

True

In [166]:
x*z*(z^2-x*5-1) in I

True

## Varieties 
### Varieties III
#### Example

In [167]:
P.<x,y> = PolynomialRing(FiniteField(7))
I = Ideal(x^2+y^2-1,x*y)
I.variety()

[{y: 0, x: 1}, {y: 0, x: 6}, {y: 1, x: 0}, {y: 6, x: 0}]

### Exercises

In [168]:
P.<x,y> = PolynomialRing(FiniteField(7))
I = Ideal(x*y+1,y**2-1)

In [169]:
x+y in I

True

$\lbrace 1 \rbrace$ is **not** an ideal since it doesn't include the $0$ element.  $\lbrace x+1,y-2 \rbrace$ is not an ideal because it doesn't include the $0$ element

In [172]:
Ideal( P(1) ) == Ideal( x+1,x-1,y-2)

True

In [174]:
Ideal(x+1,y-1).variety()

[{y: 1, x: 6}]

In [176]:
solve([x+y==P(0),x+P(1)==P(0)],x,y)

TypeError: (1, x) is not a valid variable.

In [177]:
x,y=var('x y')
solve([x+y==0,x+1==0],x,y)

[[x == -1, y == 1]]

## Gröbner Bases
### Definition II

In [179]:
R.<x> = PolynomialRing(FiniteField(7))
f=x^2+6

In [180]:
I=Ideal(map(P,[R.random_element() * f for _ in range(5)]))

In [181]:
I.groebner_basis()

[x^2 - 1]

EY:20170311 What is `map` doing?

In [182]:
map(P,[R.random_element() * f for _ in range(5)])

[-x^4 + x^3 - 3*x^2 - x - 3,
 x^4 + x^3 + x^2 - x - 2,
 3*x^4 + 2*x^3 + x^2 - 2*x + 3,
 2*x^4 - x^3 + 3*x^2 + x + 2,
 -x^4 - x^3 + x + 1]

In [194]:
P.<x,y,z> = PolynomialRing(FiniteField(127),order='deglex')
F=Sequence([-3*y,-2*x-y-3*z+2,x+y+2*z-1])

In [195]:
F.groebner_basis()

[x - 1, y, z]

In [196]:
A,v=F.coefficient_matrix()

In [197]:
A.echelonize()

In [198]:
(A*v).T

[x - 1     y     z]

EY : 20170311 what is going on?

In [200]:
print(A); print(v)

[  1   0   0 126]
[  0   1   0   0]
[  0   0   1   0]
[x]
[y]
[z]
[1]


In [202]:
print( F.coefficient_matrix()[0] )

[  0 124   0   0]
[125 126 124   2]
[  1   1   2 126]


### Definition IV
You can use Sage to check if a list of polynomials is a Gröbner basis:

In [203]:
P.<x,y,z>=PolynomialRing(FiniteField(7),order="degrevlex")
I=Ideal([x*y^2-z,2*x^2*z-2,x*y-z+1])
I.basis_is_groebner()

False

In [204]:
I.groebner_basis()

[x^2 - x + z - 2, x*y - z + 1, y^2 - x - 2*y + 1, x*z - x - y + 1, y*z - y - z, z^2 - x - y - 2*z + 2]

### Applications I
#### Representation
Reduced Gröbner bases are unique representations of an ideal with respect to a monomial ordering. So to check that two ideals are the same, you only need to compute their reduced Gröbner bases. 

In [205]:
P.<x,y,z>=PolynomialRing(FiniteField(7))
I=Ideal(P.random_element() for _ in range(5))


In [207]:
J = Ideal(P.random_element() for _ in range(5))

In [208]:
I=Ideal(f-f((2,3,1)) for f in I.gens())
J=Ideal(f-f((2,3,1)) for f in J.gens())

In [209]:
I.groebner_basis() == J.groebner_basis()

True

In [210]:
I==J

True

In [211]:
I.gens()==J.gens()

False

In [212]:
I.groebner_basis()

[x - 2, y - 3, z - 1]

### Applications II
#### Membership
remainder $r$ of division of any $f\in P$ by $G$ is unique.  Hence, we can check whether $f\in \langle f_1,\dots, f_s\rangle$ by computing a Gröbner basis $G=(g_1,\dots, g_t)$ for $\langle f_1,\dots , f_s\rangle$ and check if \overline{f}^G =0$.

In [220]:
P.<x,y,z>=PolynomialRing(FiniteField(7))
I=Ideal(P.random_element() for _ in range(5))
I=Ideal(f-f((2,3,1)) for f in I.gens())

In [222]:
f=P.random_element()

In [223]:
f in I

False

In [224]:
f = f-f((2,3,1))

In [225]:
f in I

True

In [226]:
f.reduce(I.gens())

-3*y*z + 2*z^2 - 2*x - y

In [227]:
f.reduce(I.groebner_basis())

0

[x + 1, y - 2]

### Applications III
#### Solving 


In [230]:
P.<x,y,z> = PolynomialRing(FiniteField(7), order="lex")
I=Ideal(P.random_element() for _ in range(5))
I = Ideal(f-f((2,3,1)) for f in I.gens())

In [231]:
I.groebner_basis()

[x + z - 3, y + 2*z + 2, z^2 + 3*z + 3]

In [233]:
I=Ideal(P.random_element() for _ in range(5))
I.gens()

[-y^2 + y*z + 2*z^2, 3*x*y - 2*x + y^2 + 2*z^2 - 1, -2*x^2 - 3*x*y + x*z + 3*z^2 - 3, 3*x^2 - y^2 + 2*z^2 + 2, -x*z + y^2 + 3*y*z + z^2 - 1]

#### Exercises

In [234]:
P.<x,y>=PolynomialRing(FiniteField(7),order="degrevlex")
F=Sequence([x+1,y-2,x*y-2*x+y-2])
F.groebner_basis()

[x + 1, y - 2]

In [238]:
Ideal(F).basis_is_groebner()

True

In [241]:
Ideal([x+1,y-2,x*y-2*x+y-2]).basis_is_groebner()

True

In [239]:
Ideal( Sequence([2*x+2,3*y+1]) ).basis_is_groebner()

True

In [240]:
Ideal( [2*x+2,3*y+1] ).basis_is_groebner()

True

## Buchberger's Algorithm  
### S-Polynomials I

### S-Polynomials IV

### S-Polynomials V

### S-Polynomials VI

In [274]:
P.<x,y>=PolynomialRing(QQ,order='degrevlex')

In [275]:
f0 = x^3-2*x*y
f1=x**2*y-2*y**2+x

In [278]:
(x^3*y)//x^3*f0 - (x^3*y)//(x^2*y)*f1

-x^2

Note that `//` is the same as `.quo_rem[0]`

In [276]:
(x**3*y)//x**3

y

In [277]:
(x**3*y)//x**3 == (x**3*y).quo_rem(x**3)[0]

True

This is 
$$  
x^{\gamma} = \text{LCM}{ (\text{LM}(f), \text{LM}(g) ) }
$$ 

In [282]:
xgamma = lcm(f0.lm(), f1.lm() ); print(xgamma)

x^3*y


In [283]:
xgamma//f0.lt()*f0 - xgamma//f1.lt()*f1

-x^2

### Buchberger's Criterion II

In [284]:
f1=x**3-2*x*y
f2=x**2*y-2*y**2+x

In [285]:
def make_Spoly(f1,f2):
    """ make_Spoly : make the S-polynomial """
    xgamma = lcm(f1.lm(),f2.lm())
    Spoly  = xgamma//f1.lt()*f1 - xgamma//f2.lt()*f2
    return Spoly

In [286]:
make_Spoly(f1,f2)

-x^2

In [290]:
print( make_Spoly(f1,f2)//f1.lm() )
print( make_Spoly(f1,f2)//f2.lm() )


0
0


## Buchberger's Algorithm

In [382]:
def BuchbergerAlgo(F,polyring):
    """ BuchbergerAlgo : Buchberger's Algorithm
    INPUT
    =====
    @type F : Python list of Sage Math polynomials
    @param F : F = (f_1, \dots, f_s)
    
    @type polyring : Sage Math PolynomialRing
    """
    zeroelement = polyring(0)
    pairs=[_ for _ in combinations(F,2)]
#    NO_PAIRS = len(pairs)
    G=F
    G_2 = [] # denoted G' or Gprime in Cox, Little, O'Shea (2015)

    # first case guaranteed to run 
    while G_2 != G:
        G_2 = G
        for pair in pairs:
            p=pair[0]
            q=pair[1]
            r,a_list = DivisionAlgo(make_Spoly(p,q), G, polyring)
            if r != zeroelement:
                G.append(r)
        # END of while, break condition being when we don't add,append, anymore to G
    return G

### Optional Improvements II  

#### Definition (Buchberger's First Criterion)  
Let $f,g\in \mathbb{F}[x_1,\dots, x_n]$ with disjoint leading terms, i.e.  
$$  
\text{LCM}(\text{LM}(f), \text{LM}(g))  = \text{LM}(f) \cdot \text{LM}(g)  
$$  

Then $\overline{S(f,g)}^G =0$  

In [401]:
def BuchbergerAlgo_w_1stCond(F,polyring):
    """ BuchbergerAlgo_w_1stCond : Buchberger's Algorithm, with Buchberger's First Criterion
    INPUT
    =====
    @type F : Python list of Sage Math polynomials
    @param F : F = (f_1, \dots, f_s)
    
    @type polyring : Sage Math PolynomialRing
    """
    zeroelement = polyring(0)
    pairs=[_ for _ in combinations(F,2)]
#    NO_PAIRS = len(pairs)
    G=F
    G_2 = [] # denoted G' or Gprime in Cox, Little, O'Shea (2015)

    counter = 0 # to count how many reductions to 0 of \overline{S(f,g)}^G; you can comment this line out
    
    # first case guaranteed to run 
    while G_2 != G:
        G_2 = G
        for pair in pairs:
            p=pair[0]
            q=pair[1]
            # Buchberger's First Criterion, check, before doing the Division Algorithm, DivisionAlgo
            Criterion1_LHS = lcm(p.lm(), q.lm())
            Criterion1_RHS = p.lm() * q.lm()

            # you can comment out the next 2 lines
            if Criterion1_LHS == Criterion1_RHS:
                counter +=1
            
            if Criterion1_LHS != Criterion1_RHS:
                r,a_list = DivisionAlgo(make_Spoly(p,q), G, polyring)
            #if r != zeroelement:
                G.append(r)
        # END of while, break condition being when we don't add,append, anymore to G
    print counter
    return G

#### Exercises

1. $\mathbb{F}_7[x,y]$

In [391]:
Poly_F7__2.<x,y>=PolynomialRing(FiniteField(7),2,order="deglex")
BuchbergerAlgo_w_1stCond([x**2*y+6,x*y**2-x], Poly_F7__2)

0


[x^2*y - 1, x*y^2 - x, x^2 - y]

In [392]:
# Sage Math sanity check
Ideal([x**2*y+6,x*y**2-x]).groebner_basis()

[x^2 - y, y^2 - 1]

In [395]:
Poly_F7__2.ideal([x**2*y+6,x*y**2-x]).groebner_basis()

[x^2 - y, y^2 - 1]

In [393]:
BuchbergerAlgo([x**2*y+6,x*y**2-x], Poly_F7__2)

[x^2*y - 1, x*y^2 - x, x^2 - y]

2 . $\mathbb{F}_7[x,y,z]$

In [396]:
Poly_F7__3.<x,y,z>=PolynomialRing(FiniteField(7),3,order="deglex")
BuchbergerAlgo_w_1stCond([x**2+y,x**4+2*x**2*y+y**2+3], Poly_F7__3)

0


[x^2 + y, x^4 + 2*x^2*y + y^2 + 3, -3]

In [397]:
BuchbergerAlgo([x**2+y,x**4+2*x**2*y+y**2+3], Poly_F7__3)

[x^2 + y, x^4 + 2*x^2*y + y^2 + 3, -3]

In [398]:
Poly_F7__3.ideal([x**2+y,x**4+2*x**2*y+y**2+3]).groebner_basis()

[1]

EY : 20170312 - No solution (???)

3 . $\langle x-z^4,y-z^5\rangle$ 

In [402]:
BuchbergerAlgo_w_1stCond([x-z**4,y-z**5], Poly_F7__3)

0


[-z^4 + x, -z^5 + y, -x*z + y]

In [404]:
BuchbergerAlgo([x-z**4,y-z**5], Poly_F7__3)

[-z^4 + x, -z^5 + y, -x*z + y]

In [405]:
Poly_F7__3.ideal([x-z**4,y-z**5]).groebner_basis()

[x^4 - y^3*z, y^2*z^2 - x^3, y*z^3 - x^2, z^4 - x, x*z - y]

### Quotient Rings I

In [407]:
P.<x,y,z>=PolynomialRing(FiniteField(2))
I=sage.rings.ideal.FieldIdeal(P)
Q=P.quotient_ring(I); Q

Quotient of Multivariate Polynomial Ring in x, y, z over Finite Field of size 2 by the ideal (x^2 + x, y^2 + y, z^2 + z)

In [411]:
print(I.basis);print(I.groebner_basis())

[x^2 + x, y^2 + y, z^2 + z]
[x^2 + x, y^2 + y, z^2 + z]


In [412]:
P.<x,y,z> = BooleanPolynomialRing() # much faster!
P.defining_ideal()

Ideal (x^2 + x, y^2 + y, z^2 + z) of Multivariate Polynomial Ring in x, y, z over Finite Field of size 2

## Solving Polynomial Systems with Grobner Bases  
### Elimination Ideals I

#### Elimination Theorem II  
##### Example

In [413]:
P.<x,y,z> = PolynomialRing(FiniteField(127),order='lex')
I = sage.rings.ideal.Cyclic(P)

In [414]:
I

Ideal (x + y + z, x*y + x*z + y*z, x*y*z - 1) of Multivariate Polynomial Ring in x, y, z over Finite Field of size 127

In [415]:
J=I+sage.rings.ideal.FieldIdeal(P)

These are the so-called field polynomials, that we augment our ideal with

In [416]:
sage.rings.ideal.FieldIdeal(P)

Ideal (x^127 - x, y^127 - y, z^127 - z) of Multivariate Polynomial Ring in x, y, z over Finite Field of size 127

Observe how this Groebner basis is each only in terms of 1 less variable (univariate variable); these are 
$$  
G_l=G\cap \mathbb{F}[x_l,\dots,x_n]  
$$

In [417]:
g0,g1,g2=J.groebner_basis(); g0,g1,g2

(x + y + z, y^2 + y*z + z^2, z^3 - 1)

In [418]:
factor(g2)

(z - 19) * (z - 1) * (z + 20)

In [419]:
factor(g1(x,y,19))

(y - 1) * (y + 20)

In [420]:
factor(g0(x,1,19))

x + 20

In [421]:
all(f(107,1,19)==0 for f in I.gens())

True

In [422]:
J.variety()

[{y: 19, z: 1, x: 107},
 {y: 107, z: 1, x: 19},
 {y: 1, z: 19, x: 107},
 {y: 107, z: 19, x: 1},
 {y: 1, z: 107, x: 19},
 {y: 19, z: 107, x: 1}]

### An Example II

In [431]:
x,y=var("x,y")
p1 = (x-1)**2+(y-2)**2-3**2==0
p2 = (x+1)**2+(y-1)**2-2**2==0
sols=solve([p1,p2],x,y)

In [432]:
sols

[[x == -2/5*sqrt(5) - 1, y == 4/5*sqrt(5) + 1], [x == 2/5*sqrt(5) - 1, y == -4/5*sqrt(5) + 1]]

In [439]:
print((-2/5*sqrt(5) - 1).N());print((4/5*sqrt(5) + 1).N());print((2/5*sqrt(5) - 1).N());print((-4/5*sqrt(5) + 1).N());

-1.89442719099992
2.78885438199983
-0.105572809000084
-0.788854381999832


In [426]:
P.<x,y>=PolynomialRing(QQ,order='lex')
f1=(x-1)**2+(y-2)**2-3**2
f2=(x+1)**2+(y-1)**2-2**2
I=Ideal(f1,f2)

In [427]:
I

Ideal (x^2 - 2*x + y^2 - 4*y - 4, x^2 + 2*x + y^2 - 2*y - 2) of Multivariate Polynomial Ring in x, y over Rational Field

In [428]:
I.groebner_basis()

[x + 1/2*y + 1/2, y^2 - 2*y - 11/5]

In [429]:
I.variety()

[]

In [430]:
I.variety(CC)

[{y: -0.788854381999832, x: -0.105572809000084},
 {y: 2.78885438199983, x: -1.89442719099992}]

In [440]:
P.<z>=QQ[]

In [441]:
I.variety(NumberField(z^2-5,'a'))

[{y: -4/5*a + 1, x: 2/5*a - 1}, {y: 4/5*a + 1, x: -2/5*a - 1}]

## The $\mathbb{F}_4$ Algorithm

### Warm Up

As a warm-up, consider a linear system of equations over $\mathbb{F}_{127}[x,y,z]$  

$$  
f = 26y+52z + 62 = 0  \\
g = 54y+119z+55=0  \\
h =41x+91z + 13=0  \\
$$  

$\Longrightarrow $   
$$  
\left( \begin{matrix} 0 & 26 & 52 & 62 \\ 
0 & 54 & 119 & 55 \\
41 & 0 & 91 & 13 \end{matrix} \right) $$

For these explicit steps, I looked up [Lecture 10](http://www.calvin.edu/~scofield/courses/m231/S11/calendar/lects/lect10.pdf).  Also, see [How to show the steps of Gauss' method](https://ask.sagemath.org/question/8840/how-to-show-the-steps-of-gauss-method/) and toy implementation by [jh](https://ask.sagemath.org/users/824/jh/)

In [ ]:
# Naive Gaussian reduction
def gauss_method(M,rescale_leading_entry=False):
    """Describe the reduction to echelon form of the given matrix of rationals.

    M  matrix of rationals   e.g., M = matrix(QQ, [[..], [..], ..])
    rescale_leading_entry=False  boolean  make the leading entries to 1's

    Returns: None.  Side effect: M is reduced.  Note: this is echelon form, 
    not reduced echelon form; this routine does not end the same way as does 
    M.echelon_form().

    """
    num_rows=M.nrows()
    num_cols=M.ncols()
    print M    

    col = 0   # all cols before this are already done
    for row in range(0,num_rows): 
        # ?Need to swap in a nonzero entry from below
        while (col < num_cols
               and M[row][col] == 0): 
            for i in M.nonzero_positions_in_column(col):
                if i > row:
                    print " swap row",row+1,"with row",i+1
                    M.swap_rows(row,i)
                    print M
                    break     
            else:
                col += 1

        if col >= num_cols:
            break

        # Now guaranteed M[row][col] != 0
        if (rescale_leading_entry
           and M[row][col] != 1):
            print " take",1/M[row][col],"times row",row+1
            M.rescale_row(row,1/M[row][col])
            print M
        change_flag=False
        for changed_row in range(row+1,num_rows):
            if M[changed_row][col] != 0:
                change_flag=True
                factor=-1*M[changed_row][col]/M[row][col]
                print " take",factor,"times row",row+1,"plus row",changed_row+1 
                M.add_multiple_of_row(changed_row,row,factor)
        if change_flag:
            print M
        col +=1

In [489]:
A=Matrix( FiniteField(127),[[0,26,52,62],[0,54,119,55],[41,0,91,13]])
cA = copy(A)
cA.rescale_row(2,1/41)
print(A);print(cA);
cA.swap_rows(2,0); print(cA)

[  0  26  52  62]
[  0  54 119  55]
[ 41   0  91  13]
[  0  26  52  62]
[  0  54 119  55]
[  1   0  27  22]
[  1   0  27  22]
[  0  54 119  55]
[  0  26  52  62]


In [490]:
lcm(FiniteField(127)(54),FiniteField(127)(26))

1

In [491]:
cA.rescale_row(2,1/26);print(cA);cA.rescale_row(1,1/54);print(cA);
cA.add_multiple_of_row(2,1,-1);print(cA)

[  1   0  27  22]
[  0  54 119  55]
[  0   1   2  61]
[ 1  0 27 22]
[ 0  1 61 41]
[ 0  1  2 61]
[ 1  0 27 22]
[ 0  1 61 41]
[ 0  0 68 20]


In [492]:
cA.rescale_row(2,1/68);cA.add_multiple_of_row(1,2,66);cA.add_multiple_of_row(0,2,100);print(cA); latex(cA)

[ 1  0  0 29]
[ 0  1  0 38]
[ 0  0  1 75]


\left(\begin{array}{rrrr}
1 & 0 & 0 & 29 \\
0 & 1 & 0 & 38 \\
0 & 0 & 1 & 75
\end{array}\right)

(So) After Gaussian elimination:  
$$
f'=x+29=0  \\
g' = y + 38  \\
h' = z+75 =0 
$$  
$\Longrightarrow$ 
$$  
\left(\begin{array}{rrrr}
1 & 0 & 0 & 29 \\
0 & 1 & 0 & 38 \\
0 & 0 & 1 & 75
\end{array}\right)
$$

Thus, $x=-27$, $y=-38$, and $z=-75$

### Generalizing to Non-Linear Polynomials

Now consider 2 polynomials in $\mathbb{F}_{127}[x,y,z]$ with term ordering deglex

$$  
f=x^2+2xy-2y^2+14z^2+22z  \\
g = x^2 + xy + y^2 + z^2+x+2z  $$
$\Longrightarrow $ 
$$
\left( \begin{matrix} 1 & 2 & -2 & 14 & 0 & 22 \\
1 & 1 & 1 & 1 & 1 & 2 \end{matrix} \right)$$  

$$  
f=x^2 +4y^2 - 12z^2 + 2x-18z  \\
g'=xy+-3y^2 + 13z^2 - x + 20z   
$$  
$$
\left( \begin{matrix} 1 & 0 & 4 & -12 & 2 & - 18 \\
& 1  & - 3 & 13 & -1 & 20 \end{matrix} \right)$$

### $\mathbb{F}_4$ Algorithm I

In [504]:
Poly_F127__3.<x,y,z>=PolynomialRing(FiniteField(127),3,order='deglex')
f=x**2-2*x*y-2*y**2+14*z**2
g=x+y+2*z
pairs=[_ for _ in combinations([f,g],2)]

EY:20170313, I'm not sure if the so-called slimgb algorithm is a version of $F_4$ Algorithm - someone please work it out and confirm this (cf. [Faugère's F4 Algorithm](https://ask.sagemath.org/question/8638/faugeres-f4-algorithm/) vs. http://doc.sagemath.org/html/en/reference/polynomial_rings/sage/rings/polynomial/multi_polynomial_ideal.html, 

In [510]:
print( Ideal([f,g]).groebner_basis("singular:slimgb"))
print( Ideal([f,g]).groebner_basis("libsingular:slimgb"))


[y^2 + 8*y*z + 18*z^2, x + y + 2*z]
[y^2 + 8*y*z + 18*z^2, x + y + 2*z]


In [503]:
print( pairs[0][0].lm());print( pairs[0][1].lm()) # lm for leading monomial, it doesn't include the coefficient
print( )


x^2
x


In [515]:
make_Spoly(f,g)

-3*x*y - 2*x*z - 2*y^2 + 14*z^2

These are the so-called field polynomials, that we augment our ideal with

cf. [Term orders, Polynomials, Multivariate Polynomials and Polynomial Rings](http://doc.sagemath.org/html/en/reference/polynomial_rings/sage/rings/polynomial/term_order.html)

In [299]:
P.<x,y,z>=PolynomialRing(QQ,3,order='lex')

In [251]:
x>y

True

In [252]:
x>y^2

True

In [253]:
x>1

True

In [254]:
x^1*y^2 >y^3*z^4

True

In [255]:
x^3*y^2*z^4<x^3*y^2*z^1

False

In [265]:
print(x.dict());print(y.dict());print((y**2).dict());print((x^1*y^2).dict());print( (y**3*z**4).dict());
print((x^3*y^2*z^4).dict());print((x^3*y^2*z^1).dict())

{(1, 0, 0): 1}
{(0, 1, 0): 1}
{(0, 2, 0): 1}
{(1, 2, 0): 1}
{(0, 3, 4): 1}
{(3, 2, 4): 1}
{(3, 2, 1): 1}


In [266]:
P.<x,y,z>=PolynomialRing(QQ,3,order='deglex')

In [267]:
x>y

True

In [268]:
x>y^2*z

False

In [269]:
x>1

True

In [270]:
x^1*y^2*z^3>x^3*y^2*z^0

True

In [271]:
x^2*y*z^2>x*y^3*z

True

In [273]:
print(x.dict());print(y.dict());print((y**2*z).dict());print(P(1).dict());
print((x^1*y^2*z^3).dict());print( (x**3*y**2*z**0).dict());
print((x^2*y^1*z^2).dict());print((x^1*y^3*z^1).dict())

{(1, 0, 0): 1}
{(0, 1, 0): 1}
{(0, 2, 1): 1}
{(0, 0, 0): 1}
{(1, 2, 3): 1}
{(3, 2, 0): 1}
{(2, 1, 2): 1}
{(1, 3, 1): 1}


cf. http://icm.mcs.kent.edu/reports/1995/gb.pdf

In [300]:
g1=4*x**2*z-7*y**2
g2=x*y*z**2+3*x*z**4
G=[g1,g2]

In [292]:
from itertools import combinations

In [301]:
# Algorithm Buchberger
M = [_ for _ in combinations(G,2)]
pair = M.pop()
S = make_Spoly(pair[0],pair[1])


In [302]:
S

-3*x^2*z^4 - 7/4*y^3*z

In [383]:
lcm(g1,g2)

4*x^3*y*z^3 + 12*x^3*z^5 - 7*x*y^3*z^2 - 21*x*y^2*z^4